In [ ]:
!pip uninstall googletrans
!pip install googletrans==4.0.0rc1


In [1]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googletrans import Translator
from dotenv import load_dotenv
from tqdm import tqdm
import pandas as pd
import numpy as np
import requests
import time
import os

In [2]:
_=load_dotenv()

In [3]:
api_key = os.environ['api_key']


In [4]:
#establishing connection with YouTube
youtube = build('youtube', 'v3', developerKey = api_key)

In [5]:
#list of channels to extract
channel_list = ['blockchaindailynews', 'brainbrocrypto', 'cryptoMOC','grenadetw', 'desmondcrypto']

In [6]:
#creating a function to get the channel ids 

def get_channel_ids(youtube, channel_list):
    channel_dict = {}  # Dictionary to store channel names and IDs

    for channel in channel_list:
        # Search for the channel using the channel name
        search_response = youtube.search().list(
            q=channel,
            part='id',
            type='channel',
            maxResults=1
        ).execute()

        # Extract the channel ID from the search results
        channel_id = search_response['items'][0]['id']['channelId']
        channel_dict[channel] = channel_id
        time.sleep(1)

    return channel_dict


In [9]:
channel_dict = get_channel_ids(youtube, channel_list)
channel_dict

{'blockchaindailynews': 'UCkPc41kPIJfMI0ZCTHqybYA',
 'brainbrocrypto': 'UC1EG67JfgUKaZ4Z_l4euycw',
 'cryptoMOC': 'UCv1A3TfyyeOg5q43J9pnpOw',
 'grenadetw': 'UCKyoG3_GoNdCgg5sDoDnnNg',
 'desmondcrypto': 'UCmqS0zPC0zg4L4TiYyTiK4A'}

In [10]:
unique_channel_ids =list(channel_dict.values())
unique_channel_ids                  

['UCkPc41kPIJfMI0ZCTHqybYA',
 'UC1EG67JfgUKaZ4Z_l4euycw',
 'UCv1A3TfyyeOg5q43J9pnpOw',
 'UCKyoG3_GoNdCgg5sDoDnnNg',
 'UCmqS0zPC0zg4L4TiYyTiK4A']

In [11]:
#creating a function to extract channel statistics
def get_channel_stats(youtube, unique_channel_ids):
    all_channel_data = []
    for i in range(len(unique_channel_ids)):
        request = youtube.channels().list(
            part = 'snippet, contentDetails, statistics',
            id = unique_channel_ids)
        response = request.execute()

        for i in range(len(response['items'])):
            channel_data = dict(Channel_name = response['items'][i]['snippet']['title'], 
                            Channel_playlist =  response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
                            Subscriber_count = response['items'][i]['statistics']['subscriberCount'],
                            Video_count = response['items'][i]['statistics']['videoCount'])
            all_channel_data.append(channel_data)
            
        return (all_channel_data)

In [12]:
channel_database= get_channel_stats(youtube, unique_channel_ids)
channel_database_df = pd.DataFrame(channel_database)
channel_database_df

Channel_name          Channel_playlist Subscriber_count Video_count
0     GRENADE 手榴彈  UUKyoG3_GoNdCgg5sDoDnnNg            14900          93
1      腦哥 Chill塊鏈  UU1EG67JfgUKaZ4Z_l4euycw           139000         250
2  Desmond的Web3日記  UUmqS0zPC0zg4L4TiYyTiK4A            41000         193
3           區塊鏈日報  UUkPc41kPIJfMI0ZCTHqybYA            42500         491
4          墨山貓MØC  UUv1A3TfyyeOg5q43J9pnpOw            37500         117

In [13]:
channel_database_df.to_csv('Channel_database.csv', index=False,header=True)

In [14]:
#getting playlist containing the entire video_ids of all channels
channel_playlists = list(channel_database_df['Channel_playlist'])
print(channel_playlists)


['UUKyoG3_GoNdCgg5sDoDnnNg', 'UU1EG67JfgUKaZ4Z_l4euycw', 'UUmqS0zPC0zg4L4TiYyTiK4A', 'UUkPc41kPIJfMI0ZCTHqybYA', 'UUv1A3TfyyeOg5q43J9pnpOw']


In [15]:

#creating a function to extract all the videos from the channels
def get_all_videos(playlists):
    all_video_ids= []
    for playlist_id in playlists:
        playlist_videos = []
        next_page_token = None

        while True:
            request = youtube.playlistItems().list(
                part = 'contentDetails',
                playlistId = playlist_id,
                maxResults = 50,
                pageToken=next_page_token
            )                
            response = request.execute()

            for i in range(len(response['items'])):
                video_id = response['items'][i]['contentDetails']['videoId']
                playlist_videos.append(video_id)
                    
            next_page_token = response.get('nextPageToken')               
            if not next_page_token:
                 break  

        all_video_ids.append(playlist_videos)
    return all_video_ids
                
        

In [16]:
video_list= list(get_all_videos(channel_playlists))
video_list


[['JRfDLzLNVNg',
  'e1QSIrXcDwo',
  'hgc_8Vkw4_U',
  'hzVDiif7Of4',
  'GjnhyCtJq7o',
  '0RIHJjUFZ6k',
  'gJVlWA_USUE',
  'Oa8_NII1pcs',
  'RUVQqtA4BhM',
  'RZJZV4Tpjhc',
  'pHcvBFzZQgE',
  'VDEHTuHcTZU',
  'Jg1N29pYcPU',
  '2se_kNugp7w',
  'dwLqxqCtRDU',
  'hy9OCoxwSWM',
  'iVGFonZJbI8',
  'fseSbdtXexg',
  'jvUoawEmGXw',
  'JVH-p5e91nA',
  'OLSLUlJ3vp4',
  'fLpaZklTavE',
  '_2pGbC1zQnk',
  'DAlrfL0u7O0',
  'S6udzEqXxmo',
  'C07fKj9oWmE',
  'JR07VtEJ1oY',
  'DdQn9IBRXdQ',
  'UvB-D23Neyg',
  'Qbvh4odmZ7s',
  'UuN1kv2ltPo',
  '1k5zM9JVdGY',
  '0TkT9LjSoMQ',
  '9eSKxNauj6g',
  'l5TWNfyQmvQ',
  'kuq8z6xQlV8',
  'pSZcyBZerSo',
  'MMUtRc5XLDY',
  'fS_oB1zc4eM',
  '7rPEhRVZfjg',
  'BmLk_gDogZs',
  'YwD8rdEOs0o',
  '6J5kJ5o8Xlc',
  'VIOlMGQmZvQ',
  'nV0DeD-8l08',
  'QTQNpBj6XdQ',
  'heUaspOoYVk',
  'YvuEOkMiUGk',
  'NSrpXgECmho',
  'zkhCgOjLh8A',
  'Y_1yETooEqM',
  'LX77AolM9PA',
  '6jjSQwFi6Z4',
  'uKDVMrI8gfI',
  'S4_XknLDQkU',
  '1r2Fxm1kDy4',
  'bkCavlothfY',
  '69akWcdfuHs',
  'PDYn413rtOQ

In [23]:
all_video_list = list(np.concatenate(video_list))
len(all_video_list)

1148

In [24]:
video_database = pd.Series(all_video_list)
video_database

0       JRfDLzLNVNg
1       e1QSIrXcDwo
2       hgc_8Vkw4_U
3       hzVDiif7Of4
4       GjnhyCtJq7o
           ...     
1143    buZEkiBq58I
1144    2Z7NtQuBTO4
1145    eKT8-hefm1Y
1146    PbGrWIXnQ5c
1147    jM_nk6IICt4
Length: 1148, dtype: object

In [29]:
video_database.to_csv('Video_database.csv', index=False, header=True)

In [45]:
#creating a function to extract video data from video ids
def get_video_details(youtube,videos):
    video_details = []
    for items in videos:
        page_token = None

        while True:
            request = youtube.videos().list(
                part = 'snippet, contentDetails, statistics',
                id = items,
                maxResults = 50,
                pageToken = page_token
            )
            response = request.execute()

            video_details.extend(response['items'])
            page_token = response.get('nextPageToken')

            if not page_token:
                break

        all_details = {'video_ids': [],
            'descriptions': [],
            'titles': [],
            'view_counts': [],
            'channel_ids': [],
            'publish_dates': [],
            'tags': [],
            'thumbnails': []
        }
    
        for video in video_details:
            video_id = video['id']
            snippet = video['snippet']
            statistics = video['statistics']

            description = snippet.get('description', '')
            title = snippet.get('title', '')
            view_count = statistics.get('viewCount', '')
        #like_count = statistics.get('likeCount', '')
        #dislike_count = statistics.get('dislikeCount', '')
            channel_id = snippet.get('channelId', '')
            publish_date = snippet.get('publishedAt', '')
            tag = snippet.get('tags','')
            thumbnail = snippet['thumbnails'].get('default', '')

            all_details['video_ids'].append(video_id)
            all_details['descriptions'].append(description)
            all_details['titles'].append(title)
            all_details['view_counts'].append(view_count)
            all_details['channel_ids'].append(channel_id)
            all_details['publish_dates'].append(publish_date)
            all_details['tags'].append(tag)
            all_details['thumbnails'].append(thumbnail)

    return all_details
    

In [46]:
main_database = get_video_details(youtube,video_database)
main_database

{'video_ids': ['JRfDLzLNVNg',
  'e1QSIrXcDwo',
  'hgc_8Vkw4_U',
  'hzVDiif7Of4',
  'GjnhyCtJq7o',
  '0RIHJjUFZ6k',
  'gJVlWA_USUE',
  'Oa8_NII1pcs',
  'RUVQqtA4BhM',
  'RZJZV4Tpjhc',
  'pHcvBFzZQgE',
  'VDEHTuHcTZU',
  'Jg1N29pYcPU',
  '2se_kNugp7w',
  'dwLqxqCtRDU',
  'hy9OCoxwSWM',
  'iVGFonZJbI8',
  'fseSbdtXexg',
  'jvUoawEmGXw',
  'JVH-p5e91nA',
  'OLSLUlJ3vp4',
  'fLpaZklTavE',
  '_2pGbC1zQnk',
  'DAlrfL0u7O0',
  'S6udzEqXxmo',
  'C07fKj9oWmE',
  'JR07VtEJ1oY',
  'DdQn9IBRXdQ',
  'UvB-D23Neyg',
  'Qbvh4odmZ7s',
  'UuN1kv2ltPo',
  '1k5zM9JVdGY',
  '0TkT9LjSoMQ',
  '9eSKxNauj6g',
  'l5TWNfyQmvQ',
  'kuq8z6xQlV8',
  'pSZcyBZerSo',
  'MMUtRc5XLDY',
  'fS_oB1zc4eM',
  '7rPEhRVZfjg',
  'BmLk_gDogZs',
  'YwD8rdEOs0o',
  '6J5kJ5o8Xlc',
  'VIOlMGQmZvQ',
  'nV0DeD-8l08',
  'QTQNpBj6XdQ',
  'heUaspOoYVk',
  'YvuEOkMiUGk',
  'NSrpXgECmho',
  'zkhCgOjLh8A',
  'Y_1yETooEqM',
  'LX77AolM9PA',
  '6jjSQwFi6Z4',
  'uKDVMrI8gfI',
  'S4_XknLDQkU',
  '1r2Fxm1kDy4',
  'bkCavlothfY',
  '69akWcdfuHs',
 

In [47]:
main_database_df = pd.DataFrame(main_database)
main_database_df

video_ids                                       descriptions  \
0     JRfDLzLNVNg  本影片為虛擬貨幣詐騙系列「下集」\n\n上集影片傳送門：\nhttps://bit.ly/3...   
1     e1QSIrXcDwo  《可自行調整影片播放速度》\n\n迷因幣離製作影片當下已有很大的漲跌變化，投資者須自行注意風...   
2     hgc_8Vkw4_U  📢 大家有沒有發現，最近各大場合都出現了一家交易所的身影？它就是OKX！\n\n👉 OKX ...   
3     hzVDiif7Of4  ・00:00 前言\n・00:24 事情怎麼發生的\n・02:25 為什麼可以漲到 100 ...   
4     GjnhyCtJq7o  《可自行調整影片播放速度》\n\n・00:45 前言\n・00:30 比特幣是什麼\n・01...   
...           ...                                                ...   
1143  buZEkiBq58I  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   
1144  2Z7NtQuBTO4  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   
1145  eKT8-hefm1Y  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   
1146  PbGrWIXnQ5c  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   
1147  jM_nk6IICt4  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   

                                                 titles view_counts  \
0     虛擬貨幣詐騙手段大公開！被騙了怎麼辦？調查官經手案例居然有這麼多？ft.@腦哥 @brain...        1696   
1                                     3天賺1000倍？迷因幣要怎麼玩？        1614   
2                    【OKX 交易所完整教學】入金出金、註冊、 交易、合約、Web3錢包        4672   
3                               比特幣即將在 6/17 之前突破100萬美元？         782   
4                                     比特幣暴漲，它到底是不是避險資產？         343   
...                                                 ...         ...   
1143                                        三分鐘買虛擬貨幣教學！        7338   
1144            請問我可以從幣安轉幣到小狐狸嗎？🙀別一直問能不能從哪裡轉到哪裡了|墨山貓MOC        9326   
1145                   十種在玩NFT的人🤷‍♂️你確定要買NFT嗎？想要發行NFT嗎？        5307   
1146         OpenSea全球最大NFT市場！買NFT的必經之路?全網最詳細教學 新手一次就看懂       92638   
1147                                        區塊鏈專有名詞大百科📖       17044   

                   channel_ids         publish_dates  \
0     UCKyoG3_GoNdCgg5sDoDnnNg  2023-09-21T11:00:08Z   
1     UCKyoG3_GoNdCgg5sDoDnnNg  2023-04-22T13:48:31Z   
2     UCKyoG3_GoNdCgg5sDoDnnNg  2023-04-16T12:15:05Z   
3     UCKyoG3_GoNdCgg5sDoDnnNg  2023-03-29T12:00:11Z   
4     UCKyoG3_GoNdCgg5sDoDnnNg  2023-03-27T04:00:07Z   
...                        ...                   ...   
1143  UCv1A3TfyyeOg5q43J9pnpOw  2022-03-06T12:00:28Z   
1144  UCv1A3TfyyeOg5q43J9pnpOw  2022-02-27T11:00:03Z   
1145  UCv1A3TfyyeOg5q43J9pnpOw  2022-02-13T11:19:18Z   
1146  UCv1A3TfyyeOg5q43J9pnpOw  2022-01-30T11:00:09Z   
1147  UCv1A3TfyyeOg5q43J9pnpOw  2022-01-16T11:31:56Z   

                                                   tags  \
0     [虛擬貨幣詐騙, 加密貨幣詐騙, 虛擬貨幣交易所詐騙, 泰達幣詐騙, 加密貨幣交易所詐騙, ...   
1     [Grenade手榴彈, NFT, 加密貨幣, 區塊鏈, 加密貨幣懶人包, web3, 加密...   
2                                                         
3     [Grenade手榴彈, NFT, 加密貨幣, 區塊鏈, 加密貨幣懶人包, web3, 加密...   
4     [Grenade手榴彈, NFT, 加密貨幣, 區塊鏈, 加密貨幣懶人包, web3, 加密...   
...                                                 ...   
1143                                                      
1144                                                      
1145                                                      
1146                                                      
1147                                                      

                                             thumbnails  
0     {'url': 'https://i.ytimg.com/vi/JRfDLzLNVNg/de...  
1     {'url': 'https://i.ytimg.com/vi/e1QSIrXcDwo/de...  
2     {'url': 'https://i.ytimg.com/vi/hgc_8Vkw4_U/de...  
3     {'url': 'https://i.ytimg.com/vi/hzVDiif7Of4/de...  
4     {'url': 'https://i.ytimg.com/vi/GjnhyCtJq7o/de...  
...                                                 ...  
1143  {'url': 'https://i.ytimg.com/vi/buZEkiBq58I/de...  
1144  {'url': 'https://i.ytimg.com/vi/2Z7NtQuBTO4/de...  
1145  {'url': 'https://i.ytimg.com/vi/eKT8-hefm1Y/de...  
1146  {'url': 'https://i.ytimg.com/vi/PbGrWIXnQ5c/de...  
1147  {'url': 'https://i.ytimg.com/vi/jM_nk6IICt4/de...  

[1148 rows x 8 columns]

In [48]:
main_database_df.to_csv('Main_database.csv', index = False)

In [2]:

# creating function that can iterate through each caption from a list of cols
# output = list containing a dict of video_id along with the corresponding translated captions

def get_translated_text(dataframe, columns, text_size):
    translator = Translator()
    translated_texts = []
    for column in columns:
        for i in range(len(dataframe)):
            translated_text = ''
            text = dataframe.iloc[i][column]
            video_id = dataframe.iloc[i]['video_ids']
            try:
                if len(text) > text_size:
                    for i in tqdm(range(0, len(text), text_size)):
                        translation = translator.translate(text[i: i + text_size]).text
                        translated_text += translation
                        time.sleep(1)

                else:
                    translated_text = translator.translate(text).text
            except Exception as e:
                print(e, f"Skipping video: {video_id}")

            translated_texts.append({"video_ids": video_id, f"en_{column}": translated_text})  # use the video_id as the key and the translated_caption as the value
    return translated_texts


In [3]:

# creating a function that creates a batch and then call the get_translated_caption

def translate_text_batch(dataframe, columns, text_size, batch_size):
    batches = []
    n_rows = len(dataframe)
    n_batches = (n_rows // batch_size) + (1 if n_rows % batch_size != 0 else 0)

    for i in range(n_batches):
        start = i * batch_size
        end = min((i + 1) * batch_size, n_rows)
        batch_df = dataframe.iloc[start:end]

        try:
            translated_dict = get_translated_text(batch_df, columns, text_size)
            # translated_df = pd.DataFrame(translated_dict)
            batches += translated_dict
        except Exception as e:
            print(f"Error occurred while translating batch {i + 1}: {e}")

        print(f"Translated batch {i + 1} of {n_batches}")
        # time.sleep(sleep_time)
    # concatenate the batches into one dataframe
       # df_translated = pd.concat(batches, ignore_index=True)
    # # join the translated dataframe with the original dataframe
    # df_joined = dataframe.merge(df_translated, on='video_id')
    # # return the joined dataframe
    return pd.DataFrame(batches)



In [4]:
description_to_translate =  pd.read_csv('Main_database.csv')
description_to_translate.head(1)

video_ids                                       descriptions  \
0  JRfDLzLNVNg  本影片為虛擬貨幣詐騙系列「下集」\n\n上集影片傳送門：\nhttps://bit.ly/3...   

                                              titles  view_counts  \
0  虛擬貨幣詐騙手段大公開！被騙了怎麼辦？調查官經手案例居然有這麼多？ft.@腦哥 @brain...         1696   

                channel_ids         publish_dates  \
0  UCKyoG3_GoNdCgg5sDoDnnNg  2023-09-21T11:00:08Z   

                                                tags  \
0  ['虛擬貨幣詐騙', '加密貨幣詐騙', '虛擬貨幣交易所詐騙', '泰達幣詐騙', '加密...   

                                          thumbnails  
0  {'url': 'https://i.ytimg.com/vi/JRfDLzLNVNg/de...

In [ ]:

if __name__ == '__main__':
    testX = description_to_translate
    columns = ['descriptions']
    batch_size = 50
    sleep_time = 10
    text_size = 2000
    testX_translated = translate_text_batch(testX, columns, text_size, batch_size)
    combined_df = pd.merge(testX, testX_translated, how='outer', left_on='video_ids', right_on='video_ids')
    

In [64]:
combined_df.to_csv('Translated_descriptions.csv', index = False)

In [ ]:

if __name__ == '__main__':
    path = "Translated_descriptions.csv"    
    titles_to_translate = pd.read_csv(path)
    df = titles_to_translate
    columns = ['titles']    
    batch_size = 50
    sleep_time = 1   
    text_size = 2000
    df_translated = translate_text_batch(df, columns, text_size, batch_size)
    translated_titles_df = pd.merge(df, df_translated, how = 'outer', left_on='video_ids', right_on='video_ids')
    
translated_titles_df


In [59]:
translated_titles_df

video_ids                                       descriptions  \
0     JRfDLzLNVNg  本影片為虛擬貨幣詐騙系列「下集」\n\n上集影片傳送門：\nhttps://bit.ly/3...   
1     e1QSIrXcDwo  《可自行調整影片播放速度》\n\n迷因幣離製作影片當下已有很大的漲跌變化，投資者須自行注意風...   
2     hgc_8Vkw4_U  📢 大家有沒有發現，最近各大場合都出現了一家交易所的身影？它就是OKX！\n\n👉 OKX ...   
3     hzVDiif7Of4  ・00:00 前言\n・00:24 事情怎麼發生的\n・02:25 為什麼可以漲到 100 ...   
4     GjnhyCtJq7o  《可自行調整影片播放速度》\n\n・00:45 前言\n・00:30 比特幣是什麼\n・01...   
...           ...                                                ...   
1143  buZEkiBq58I  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   
1144  2Z7NtQuBTO4  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   
1145  eKT8-hefm1Y  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   
1146  PbGrWIXnQ5c  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   
1147  jM_nk6IICt4  我們的官方Line上線囉😽\n趕緊聯繫我們了解虛擬貨幣吧\nhttps://lin.ee/y...   

                                                 titles  view_counts  \
0     虛擬貨幣詐騙手段大公開！被騙了怎麼辦？調查官經手案例居然有這麼多？ft.@腦哥 @brain...         1696   
1                                     3天賺1000倍？迷因幣要怎麼玩？         1614   
2                    【OKX 交易所完整教學】入金出金、註冊、 交易、合約、Web3錢包         4672   
3                               比特幣即將在 6/17 之前突破100萬美元？          782   
4                                     比特幣暴漲，它到底是不是避險資產？          343   
...                                                 ...          ...   
1143                                        三分鐘買虛擬貨幣教學！         7338   
1144            請問我可以從幣安轉幣到小狐狸嗎？🙀別一直問能不能從哪裡轉到哪裡了|墨山貓MOC         9326   
1145                   十種在玩NFT的人🤷‍♂️你確定要買NFT嗎？想要發行NFT嗎？         5307   
1146         OpenSea全球最大NFT市場！買NFT的必經之路?全網最詳細教學 新手一次就看懂        92638   
1147                                        區塊鏈專有名詞大百科📖        17044   

                   channel_ids         publish_dates  \
0     UCKyoG3_GoNdCgg5sDoDnnNg  2023-09-21T11:00:08Z   
1     UCKyoG3_GoNdCgg5sDoDnnNg  2023-04-22T13:48:31Z   
2     UCKyoG3_GoNdCgg5sDoDnnNg  2023-04-16T12:15:05Z   
3     UCKyoG3_GoNdCgg5sDoDnnNg  2023-03-29T12:00:11Z   
4     UCKyoG3_GoNdCgg5sDoDnnNg  2023-03-27T04:00:07Z   
...                        ...                   ...   
1143  UCv1A3TfyyeOg5q43J9pnpOw  2022-03-06T12:00:28Z   
1144  UCv1A3TfyyeOg5q43J9pnpOw  2022-02-27T11:00:03Z   
1145  UCv1A3TfyyeOg5q43J9pnpOw  2022-02-13T11:19:18Z   
1146  UCv1A3TfyyeOg5q43J9pnpOw  2022-01-30T11:00:09Z   
1147  UCv1A3TfyyeOg5q43J9pnpOw  2022-01-16T11:31:56Z   

                                                   tags  \
0     ['虛擬貨幣詐騙', '加密貨幣詐騙', '虛擬貨幣交易所詐騙', '泰達幣詐騙', '加密...   
1     ['Grenade手榴彈', 'NFT', '加密貨幣', '區塊鏈', '加密貨幣懶人包'...   
2                                                   NaN   
3     ['Grenade手榴彈', 'NFT', '加密貨幣', '區塊鏈', '加密貨幣懶人包'...   
4     ['Grenade手榴彈', 'NFT', '加密貨幣', '區塊鏈', '加密貨幣懶人包'...   
...                                                 ...   
1143                                                NaN   
1144                                                NaN   
1145                                                NaN   
1146                                                NaN   
1147                                                NaN   

                                             thumbnails     video_id  \
0     {'url': 'https://i.ytimg.com/vi/JRfDLzLNVNg/de...  JRfDLzLNVNg   
1     {'url': 'https://i.ytimg.com/vi/e1QSIrXcDwo/de...  e1QSIrXcDwo   
2     {'url': 'https://i.ytimg.com/vi/hgc_8Vkw4_U/de...  hgc_8Vkw4_U   
3     {'url': 'https://i.ytimg.com/vi/hzVDiif7Of4/de...  hzVDiif7Of4   
4     {'url': 'https://i.ytimg.com/vi/GjnhyCtJq7o/de...  GjnhyCtJq7o   
...                                                 ...          ...   
1143  {'url': 'https://i.ytimg.com/vi/buZEkiBq58I/de...  buZEkiBq58I   
1144  {'url': 'https://i.ytimg.com/vi/2Z7NtQuBTO4/de...  2Z7NtQuBTO4   
1145  {'url': 'https://i.ytimg.com/vi/eKT8-hefm1Y/de...  eKT8-hefm1Y   
1146  {'url': 'https://i.ytimg.com/vi/PbGrWIXnQ5c/de...  PbGrWIXnQ5c   
1147  {'url': 'https://i.ytimg.com/vi/jM_nk6IICt4/de...  jM_nk6IICt4   

                               

In [ ]:
translated_titles_df.to_csv('Translated_descriptions_titles.csv', index = False)

In [ ]:

if __name__ == '__main__':
    path = 'Translated_descriptions_titles.csv'
    tags_to_translate = pd.read_csv(path)
    tag_df = tags_to_translate
    columns = ['tags']
    batch_size = 50
    sleep_time = 1
    text_size = 2000
    df_translated = translate_text_batch(tag_df, columns, text_size, batch_size)
    translated_tags_df = pd.merge(tag_df, df_translated, left_on='video_ids', right_on='video_ids')
    
translated_tags_df


In [ ]:
translated_tags_df.to_csv('Main_translated_database.csv', index = False)